In [1]:
import pandas as pd
import pyam
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import aneris

# Cleaning Data

In [2]:
ifs_path = '../../../data/SOD/model_results/raw/IFs_R2_notes.xlsx'

In [3]:
def round_to_0_1_percent(value):
    if value == 0:
        return 0
    # Determine the number of significant digits to round to
    magnitude = np.floor(np.log10(abs(value))) - 2
    rounding_factor = 10 ** magnitude
    return np.round(value / rounding_factor) * rounding_factor

In [4]:
df_unclean = pd.read_excel(ifs_path)
df_unclean = df_unclean.dropna()
df_unclean['Model'] = df_unclean['Model'].replace('Ifs 8.26', 'IFs 8.26')
df_unclean['Scenario'] = df_unclean['Scenario'].replace({'GEOLife': 'LIFE-TP-v2',
                                                   'GEOTech': 'TECH-TP-v2',
                                                   'GEOREF': 'REF-v2'})
df_unclean['Region'] = df_unclean['Region'].replace({'UNEPAfrica': 'Africa (UN-R5)',
                                                'UNEPAPac': 'Asia and the Pacific (UN-R5)',
                                                'UNEPEEurope': 'Eastern Europe (UN-R5)',
                                                'UNEPLAC': 'Latin America and Caribbean (UN-R5)',
                                                'UNEPWEurope': 'Western Europe and Other States (UN-R5)',
                                                })

# Create a boolean mask for rows containing '|Male' or '|Female'
sex_mask = df_unclean['Variable'].str.contains(r'\|Male|\|Female')
# Filter out those rows
df_unclean = df_unclean[~sex_mask]

df_unclean['Variable'] = df_unclean['Variable'].replace({
    'Popuilation|Calories Per Capita|Coefficient of Variation': 'Population|Calories Per Capita|Coefficient of Variation',
    'Popilation|Electricity Access (Share)': 'Population|Electricity Access (Share)',
    'Population|Malnourished Population': 'Population|Malnourished',
    'Health Indicator|Protien Per Capita|Daily': 'Health Indicator|Protein Per Capita|Daily'
})

# Create a boolean mask for rows where the 'unit' column contains '%'
ratio_mask = df_unclean['Unit'] == 'Ratio'
# Apply rounding to the 'value' column for rows where the 'unit' column is '%'
df_unclean.loc[ratio_mask, df_unclean.columns[5:]] = df_unclean.loc[ratio_mask, df_unclean.columns[5:]] * 100

df_unclean['Unit'] = df_unclean['Unit'].replace({
    'People': 'million',
    'Mil People': 'million',
    'Population|Malnourished Population':'Population|Malnourished',
    'Thousand 2017$': 'thousand USD_2017/yr',
    'Thousand dollars': 'thousand USD_2017/yr',
    'Percent': '%',
    'Ratio': '%',
    'Calories Per Capita': "kcal/cap",
    'Thousands of Deaths': 'thousand deaths'
})

# Create a boolean mask for rows where the 'unit' column contains '%'
percent_mask = df_unclean['Unit'] == '%'
# Apply rounding to the 'value' column for rows where the 'unit' column is '%'
df_unclean.loc[percent_mask, df_unclean.columns[5:]] = df_unclean.loc[percent_mask, df_unclean.columns[5:]].round()

df_unclean.loc[:, df_unclean.columns[5:]] = df_unclean.loc[:, df_unclean.columns[5:]].applymap(round_to_0_1_percent)

df_unclean

C:\Users\maycocks\AppData\Local\Temp\2\ipykernel_6008\771386026.py:48: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_unclean.loc[:, df_unclean.columns[5:]] = df_unclean.loc[:, df_unclean.columns[5:]].applymap(round_to_0_1_percent)


,Model,Scenario,Region,Variable,Unit,2019,2020,2021,2022,2023,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
0,IFs 8.26,LIFE-TP-v2,Africa (UN-R5),Health Indicator|Calories Per Capita,kcal/cap,2560.0,2550.0,2560.0,2590.0,2590.0,...,3480.0,3480.0,3490.0,3500.0,3520.0,3530.0,3540.0,3550.0,3560.0,3570.0
1,IFs 8.26,REF-v2,Africa (UN-R5),Health Indicator|Calories Per Capita,kcal/cap,2560.0,2550.0,2560.0,2590.0,2590.0,...,3460.0,3470.0,3480.0,3490.0,3500.0,3510.0,3520.0,3530.0,3540.0,3550.0
2,IFs 8.26,TECH-TP-v2,Africa (UN-R5),Health Indicator|Calories Per Capita,kcal/cap,2560.0,2550.0,2560.0,2590.0,2590.0,...,3480.0,3490.0,3500.0,3510.0,3520.0,3530.0,3540.0,3550.0,3560.0,3570.0
3,IFs 8.26,LIFE-TP-v2,Asia and the Pacific (UN-R5),Health Indicator|Calories Per Capita,kcal/cap,2860.0,2850.0,2870.0,2890.0,2910.0,...,3630.0,3630.0,3640.0,3650.0,3650.0,3660.0,3660.0,3670.0,3680.0,3680.0
4,IFs 8.26,REF-v2,Asia and the Pacific (UN-R5),Health Indicator|Calories Per Capita,kcal/cap,2860.0,2850.0,2870.0,2890.0,2910.0,...,3650.0,3660.0,3660.0,3670.0,3670.0,3680.0,3680.0,3690.0,3690.0,3700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219,IFs 8.26,REF-v2,Western Europe and Other States (UN-R5),Population|Safe Water Access,%,98.0,98.0,98.0,98.0,98.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
1220,IFs 8.26,TECH-TP-v2,Western Europe and Other States (UN-R5),Population|Safe Water Access,%,98.0,98.0,98.0,99.0,99.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
1221,IFs 8.26,LIFE-TP-v2,World,Population|Safe Water Access,%,69.0,71.0,74.0,77.0,78.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
1222,IFs 8.26,REF-v2,World,Population|Safe Water Access,%,69.0,68.0,69.0,70.0,70.0,...,97.0,97.0,97.0,97.0,97.0,97.0,98.0,98.0,98.0,98.0


In [5]:
comm_vars = [
    'Health Indicator|OthCommumDis Deaths|Total',
    'Health Indicator|AIDS Deaths|Total',
    'Health Indicator|Diarrhea Deaths|Total',
    'Health Indicator|Malaria Deaths|Total',
    'Health Indicator|RespInfec Deaths|Total',]

non_commu_vars = [
'Health Indicator|MentalHealth Deaths|Total',
'Health Indicator|MaligNeoPl Deaths|Total',
'Health Indicator|CardioVasc Deaths|Total',
'Health Indicator|Digestive Deaths|Total',
'Health Indicator|Respiratory Deaths|Total',
'Health Indicator|OtherNonComm Deaths|Total',
'Health Indicator|Diabetes Deaths|Total',
'Health Indicator|MentalHealth Deaths|Total',
]

injury_vars = [
'Health Indicator|TrafficAcc Deaths|Total',
'Health Indicator|UnIntInj Deaths|Total',
'Health Indicator|IntInj Deaths|Total',
]



df_mid_clean = pyam.IamDataFrame(df_unclean)

df_mid_clean.aggregate("Health|Deaths|Communicable", 
                       components=comm_vars, 
                        append=True)

df_mid_clean.aggregate("Health|Deaths|Non-Communicable", 
                       components=non_commu_vars, 
                        append=True)

df_mid_clean.aggregate("Health|Deaths|Injuries", 
                       components=injury_vars, 
                        append=True)

df_mid_clean.aggregate("Health|Deaths", 
                       components=['Health Indicator|Total Deaths|Total'], 
                        append=True)

vars_to_viz = [
    'Population|Calories Per Capita|Coefficient of Variation',
    'Labor Productivity',
    'Labor Productivity|Agriculture',
    'Labor Productivity|Energy',
    'Labor Productivity|Materials',
    'Labor Productivity|Manufactures',
    'Labor Productivity|Services',
    'Labor Productivity|ICTech',
    'Population|Malnourished',
    'Population|Poverty Under $2.15',
    'Population|Shared Sanitation Access',
    'Population|Safe Water Access',
    "Health|Deaths",
    "Health|Deaths|Injuries",
    "Health|Deaths|Non-Communicable",
    "Health|Deaths|Communicable",]

df_mid_clean = df_mid_clean.filter(variable=vars_to_viz, year=[2020, 2030, 2040, 2050, 2060])

pandas_df = df_mid_clean.timeseries()
pandas_df = pandas_df.applymap(round_to_0_1_percent)
clean_df = pyam.IamDataFrame(pandas_df)

clean_df.timeseries()

C:\Users\maycocks\AppData\Local\Temp\2\ipykernel_6008\2789885707.py:66: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pandas_df = pandas_df.applymap(round_to_0_1_percent)


2020  \
model    scenario   region         variable                                           unit                            
IFs 8.26 LIFE-TP-v2 Africa (UN-R5) Health|Deaths                                      thousand deaths       8620.00   
                                   Health|Deaths|Communicable                         thousand deaths       4400.00   
                                   Health|Deaths|Injuries                             thousand deaths        639.00   
                                   Health|Deaths|Non-Communicable                     thousand deaths       3580.00   
                                   Labor Productivity                                 thousand USD_2017/yr     4.81   
...                                                                                                             ...   
         TECH-TP-v2 World          Population|Calories Per Capita|Coefficient of V... %                       24.00   
                                   Population|Malnourished                            million                615.00   
                                   Population|Poverty Under $2.15                     million                785.00   
                                   Population|Safe Water Access                       %                       71.00   
                                   Population|Shared Sanitation Access                %                       59.00   

                                                                                                              2030  \
model    scenario   region         variable                                           unit                           
IFs 8.26 LIFE-TP-v2 Africa (UN-R5) Health|Deaths                                      thousand deaths       9510.0   
                                   Health|Deaths|Communicable                         thousand deaths       3960.0   
                                   Health|Deaths|Injuries                             thousand deaths        847.0   
                                   Health|Deaths|Non-Communicable                     thousand deaths       4700.0   
                                   Labor Productivity                                 thousand USD_2017/yr     5.3   
...                                                                                                            ...   
         TECH-TP-v2 World          Population|Calories Per Capita|Coefficient of V... %                       19.0   
                                   Population|Malnourished                            million                298.0   
                                   Population|Poverty Under $2.15                     million                655.0   
                                   Population|Safe Water Access                       %                       86.0   
                                   Population|Shared Sanitation Access                %                       80.0   

                                                                                                                2040  \
model    scenario   region         variable                                           unit                             
IFs 8.26 LIFE-TP-v2 Africa (UN-R5) Health|Deaths                                      thousand deaths       10000.00   
                                   Health|Deaths|Communicable                         thousand deaths        3130.00   
                                   Health|Deaths|Injuries                             thousand deaths        1050.00   
                                   Health|Deaths|Non-Communicable                     thousand deaths        5860.00   
                                   Labor Productivity                                 thousand USD_2017/yr      7.03   
...                                                                                                              ...   
         TECH-TP-v2 World          Population|Calories Per Capita|Co

# Harmonize

In [6]:
# Reset index for a specific level
methods = clean_df.timeseries().reset_index(level='model', drop=True) # filter(variable='Land Cover|Cropland').
# The offset methods ensure internal regional consistency better than the ratio methods
methods['method'] = 'reduce_offset_2030'
methods = methods['method']
methods

scenario    region          variable                                                 unit                
LIFE-TP-v2  Africa (UN-R5)  Health|Deaths                                            thousand deaths         reduce_offset_2030
                            Health|Deaths|Communicable                               thousand deaths         reduce_offset_2030
                            Health|Deaths|Injuries                                   thousand deaths         reduce_offset_2030
                            Health|Deaths|Non-Communicable                           thousand deaths         reduce_offset_2030
                            Labor Productivity                                       thousand USD_2017/yr    reduce_offset_2030
                                                                                                                    ...        
TECH-TP-v2  World           Population|Calories Per Capita|Coefficient of Variation  %                       reduce_offset_203

In [7]:
df_copy = clean_df.timeseries()
# Remove the model level from index
df_copy = df_copy.reset_index(level='model', drop=True)
df_copy.columns = df_copy.columns.astype(str)

hist_filtered = clean_df.filter(scenario='REF-v2', year=[2020]).timeseries()
hist_filtered = hist_filtered.copy().reset_index(level='model', drop=True)
hist_filtered.columns = hist_filtered.columns.astype(str)

def change_first_level_index_value(df, old_value, new_value):
    new_index = df.index.to_frame()  # Convert index to DataFrame
    new_index.loc[new_index['scenario'] == old_value, 'scenario'] = new_value  # Change the values
    new_df = df.copy()
    new_df.index = pd.MultiIndex.from_frame(new_index)  # Convert back to MultiIndex
    return new_df

# Change 'A' to 'C' in the first level of the MultiIndex
life_hist = change_first_level_index_value(hist_filtered, 'REF-v2', 'LIFE-TP-v2')
tech_hist = change_first_level_index_value(hist_filtered, 'REF-v2', 'TECH-TP-v2')

hist_conc = pd.concat([hist_filtered, life_hist, tech_hist])

In [8]:
# Harmonize 
h = aneris.harmonize.Harmonizer(df_copy, hist_conc, config={'harmonize_year': '2020'})
h.methods_used
harmonized = h.harmonize(methods) # methods_copy
harmonized
new_level = 'IFs 8.26'
df_index = harmonized.index.to_frame()  # Convert the MultiIndex to a DataFrame
df_index['model'] = new_level  # Add the new level
df_index = df_index[['model', 'scenario', 'region', 'variable', 'unit']]
new_index = pd.MultiIndex.from_frame(df_index)  # Create a new MultiIndex with the added level
harmonized.index = new_index
harmonized

c:\PBLprogs\miniforge3\lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\PBLprogs\miniforge3\lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\PBLprogs\miniforge3\lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\PBLprogs\miniforge3\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\PBLprogs\miniforge3\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
INFO:root:Harmonizing with reduce_offset_2030


2020  \
model    scenario   region         variable                                           unit                            
IFs 8.26 LIFE-TP-v2 Africa (UN-R5) Health|Deaths                                      thousand deaths       8620.00   
                                   Health|Deaths|Communicable                         thousand deaths       4400.00   
                                   Health|Deaths|Injuries                             thousand deaths        639.00   
                                   Health|Deaths|Non-Communicable                     thousand deaths       3580.00   
                                   Labor Productivity                                 thousand USD_2017/yr     4.81   
...                                                                                                             ...   
         TECH-TP-v2 World          Population|Calories Per Capita|Coefficient of V... %                       24.00   
                                   Population|Malnourished                            million                634.00   
                                   Population|Poverty Under $2.15                     million                790.00   
                                   Population|Safe Water Access                       %                       68.00   
                                   Population|Shared Sanitation Access                %                       55.00   

                                                                                                              2030  \
model    scenario   region         variable                                           unit                           
IFs 8.26 LIFE-TP-v2 Africa (UN-R5) Health|Deaths                                      thousand deaths       9510.0   
                                   Health|Deaths|Communicable                         thousand deaths       3960.0   
                                   Health|Deaths|Injuries                             thousand deaths        847.0   
                                   Health|Deaths|Non-Communicable                     thousand deaths       4700.0   
                                   Labor Productivity                                 thousand USD_2017/yr     5.3   
...                                                                                                            ...   
         TECH-TP-v2 World          Population|Calories Per Capita|Coefficient of V... %                       19.0   
                                   Population|Malnourished                            million                298.0   
                                   Population|Poverty Under $2.15                     million                655.0   
                                   Population|Safe Water Access                       %                       86.0   
                                   Population|Shared Sanitation Access                %                       80.0   

                                                                                                                2040  \
model    scenario   region         variable                                           unit                             
IFs 8.26 LIFE-TP-v2 Africa (UN-R5) Health|Deaths                                      thousand deaths       10000.00   
                                   Health|Deaths|Communicable                         thousand deaths        3130.00   
                                   Health|Deaths|Injuries                             thousand deaths        1050.00   
                                   Health|Deaths|Non-Communicable                     thousand deaths        5860.00   
                                   Labor Productivity                                 thousand USD_2017/yr      7.03   
...                                                                                                              ...   
         TECH-TP-v2 World          Population|Calories Per Capita|Co

# For these indicators, only for the TPs, make a new variable presenting the results as a “difference relative to REF”
- Health|Deaths|Communicable
- Health|Deaths|Non-Communicble
- Health|Deaths|Injuries
- Health|Deaths|Total

In [9]:
harmonized_df = pyam.IamDataFrame(harmonized)
tech_diff = harmonized_df.filter(variable=[
    'Health*']).subtract("TECH-TP-v2", "REF-v2", "TECH-TP-v2", axis='scenario', ignore_units='thousand deaths')
tech_diff = tech_diff.rename(variable={
    'Health|Deaths':'Health|Deaths (Difference Relative to REF)',
    'Health|Deaths|Communicable':'Health|Deaths|Communicable (Difference Relative to REF)',
    'Health|Deaths|Injuries':'Health|Deaths|Injuries (Difference Relative to REF)',
    'Health|Deaths|Non-Communicable':'Health|Deaths|Non-Communicable (Difference Relative to REF)',
})

life_diff = harmonized_df.filter(variable=[
    'Health*']).subtract("LIFE-TP-v2", "REF-v2", "LIFE-TP-v2", axis='scenario', ignore_units='thousand deaths')
life_diff = life_diff.rename(variable={
    'Health|Deaths':'Health|Deaths (Difference Relative to REF)',
    'Health|Deaths|Communicable':'Health|Deaths|Communicable (Difference Relative to REF)',
    'Health|Deaths|Injuries':'Health|Deaths|Injuries (Difference Relative to REF)',
    'Health|Deaths|Non-Communicable':'Health|Deaths|Non-Communicable (Difference Relative to REF)',
})

life_diff.timeseries()

2020  \
model    scenario   region                                  variable                                           unit                    
IFs 8.26 LIFE-TP-v2 Africa (UN-R5)                          Health|Deaths (Difference Relative to REF)         thousand deaths   0.0   
                                                            Health|Deaths|Communicable (Difference Relative... thousand deaths   0.0   
                                                            Health|Deaths|Injuries (Difference Relative to ... thousand deaths   0.0   
                                                            Health|Deaths|Non-Communicable (Difference Rela... thousand deaths   0.0   
                    Asia and the Pacific (UN-R5)            Health|Deaths (Difference Relative to REF)         thousand deaths   0.0   
                                                            Health|Deaths|Communicable (Difference Relative... thousand deaths   0.0   
                                                            Health|Deaths|Injuries (Difference Relative to ... thousand deaths   0.0   
                                                            Health|Deaths|Non-Communicable (Difference Rela... thousand deaths   0.0   
                    Eastern Europe (UN-R5)                  Health|Deaths (Difference Relative to REF)         thousand deaths   0.0   
                                                            Health|Deaths|Communicable (Difference Relative... thousand deaths   0.0   
                                                            Health|Deaths|Injuries (Difference Relative to ... thousand deaths   0.0   
                                                            Health|Deaths|Non-Communicable (Difference Rela... thousand deaths   0.0   
                    Latin America and Caribbean (UN-R5)     Health|Deaths (Difference Relative to REF)         thousand deaths   0.0   
                                                            Health|Deaths|Communicable (Difference Relative... thousand deaths   0.0   
                                                            Health|Deaths|Injuries (Difference Relative to ... thousand deaths   0.0   
                                                            Health|Deaths|Non-Communicable (Difference Rela... thousand deaths   0.0   
                    Western Europe and Other States (UN-R5) Health|Deaths (Difference Relative to REF)         thousand deaths   0.0   
                                                            Health|Deaths|Communicable (Difference Relative... thousand deaths   0.0   
                                                            Health|Deaths|Injuries (Difference Relative to ... thousand deaths   0.0   
                                                            Health|Deaths|Non-Communicable (Difference Rela... thousand deaths   0.0   
                    World                                   Health|Deaths (Difference Relative to REF)         thousand deaths   0.0   
                                                            Health|Deaths|Communicable (Difference Relative... thousand deaths   0.0   
                                                            Health|Deaths|Injuries (Difference Relative to ... thousand deaths   0.0   
                                                            Health|Deaths|Non-Communicable (Difference Rela... thousand deaths   0.0   

                                                                                                                                 2030  \
model    scenario   region                                  variable                                           unit                     
IFs 8.26 LIFE-TP-v2 Africa (UN-R5)                          Health|Deaths (Difference Relative to REF)         thousand deaths -340.0   
                                                            Health|Deaths|Communicable (Difference Relative... thousand deaths -180.0   
                                           

# Plotting

In [10]:
def plot_data_3_x_3_figures(df, title_dict=None, path_dict=None, variables=None,):

    # Define the color palette for scenarios
    scenario_colors = {
        "REF": "black",
        "LIFE-TP": "orange",
        "TECH-TP": "magenta",
        "REF-v2": "black",
        "LIFE-TP-v2": "orange",
        "TECH-TP-v2": "magenta",
    }

    df_plotting = df.filter(year=[2010, 2015, 2020, 2025, 2030, 2035, 2040, 2045, 2050]).timeseries().reset_index()
    df_compare_data = pd.melt(df_plotting, id_vars=['model',
                                'scenario',
                                'region',
                                'variable',
                                'unit'
                                    ], var_name='year', value_name='value')
    df_compare_data['year'] = df_compare_data['year'].astype(int)

    if variables is None:
        variables = df.variable

    for variable in variables:
        df_var = df_compare_data[df_compare_data["variable"] == variable]
        # Get unique regions for creating subplots
        unique_regions = df_var['region'].unique()

        # Create subplots based on the number of unique regions
        fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))

        # Initialize lists to store handles and labels for the legend
        legend_handles = []
        legend_labels = []

        # Iterate through each region and plot in respective subplot
        for idx, region in enumerate(unique_regions):
            row_idx = idx // 3
            col_idx = idx % 3
            
            ax = axes[row_idx, col_idx]  # Select the current subplot
            
            region_data = df_var[df_var['region'] == region]  # Filter data for the current region
            
            # Plot lineplots for each region
            lineplot = sns.lineplot(data=region_data, 
                                    x="year", 
                                    y="value", 
                                    hue="scenario", 
                                    # style='model', 
                                    palette=scenario_colors, 
                                    linewidth=2.5, 
                                    ax=ax)
            
            # Customize ticks and labels for the current subplot
            ax.set_ylabel(region_data["unit"].iloc[0], fontsize=14)  # Set the y-axis label to the unit
            # Set y-axis lower limit to 0
            ax.set_ylim(min(0,ax.get_ylim()[0]), ax.get_ylim()[1])
            ax.set_xlabel('', fontsize=1)
            ax.set_title(f"{region}", fontsize=14)  # Set the title to the region
            ax.set_xticks([2010, 2020, 2030, 2040, 2050])
            ax.set_xticklabels([2010, 2020, 2030, 2040, 2050], 
                            rotation=45, fontsize=14)
            
            # Increase fontsize of y-axis tick labels
            ax.tick_params(axis='y', labelsize=14)

            # Add the lineplot to the legend manually
            legend_handles.append(lineplot)
            legend_labels.append(region)
            # Disable the legend for individual subplots
            ax.legend().set_visible(False)
            ax.grid(True)

            # Add a gray background for the 'World' region subplot
            if region == 'World':
                ax.set_facecolor(color='#ededed')

        # Show legend and grid for the current subplot
        if title_dict:
            fig.suptitle(f"{title_dict[variable]}", fontsize=16)
        else:
            fig.suptitle(variable, fontsize=16)

        handles, labels = ax.get_legend_handles_labels()
        fig.legend(handles, labels, title="Legend", bbox_to_anchor=(1.12, 0.55))
        plt.tight_layout()  # Adjust subplots to prevent overlap
        
        if path_dict:
            variable_name = variable.replace(" > 25 μg/m3", "").replace("|", "_").replace(" ", "_").replace("/", "")
            plt.savefig(f"../../../plots/SOD/chpt_20/Regional/{path_dict[variable]}/{variable_name}.png", bbox_inches='tight')
            plt.close()

In [11]:
def plot_data_individual_figures(df, title_dict=None, path_dict=None, variables=None,):
    # Define the color palette for scenarios
    scenario_colors = {
        "REF": "black",
        "LIFE-TP": "orange",
        "TECH-TP": "magenta",
        "REF-v2": "black",
        "LIFE-TP-v2": "orange",
        "TECH-TP-v2": "magenta",
    }

    regions_dict = {
        'Africa (UN-R5)': 'Africa',
        'Asia and the Pacific (UN-R5)': 'AatP',
        'Eastern Europe (UN-R5)': 'EE',
        'Latin America and Caribbean (UN-R5)': 'LaaC',
        'Western Europe and Other States (UN-R5)': 'WeoS'
                }
    
    if variables is None:
            variables = df.variable

    for region in list(regions_dict.keys()):
        df_plotting = df.filter(year=[2010, 2015, 2020, 2025, 2030, 2035, 2040, 2045, 2050], region=region).timeseries().reset_index()

        
        df_selected = df_plotting[df_plotting['variable'].isin(variables)]

        df_compare_data = pd.melt(df_selected, id_vars=['model',
                                    'scenario',
                                    'region',
                                    'variable',
                                    'unit'
                                        ], var_name='year', value_name='value')
        df_compare_data['year'] = df_compare_data['year'].astype(int)

        

        for variable in variables: # ,variables
            # Use Seaborn to create the plot with markers and lines
            data = df_compare_data[df_compare_data["variable"] == variable]

            fig, ax = plt.subplots(figsize=(10, 6))  # Set figure size
            sns.lineplot(
                data=data,
                x="year",
                y="value",
                hue="scenario",
                # style="model",
                dashes=True,
                palette=scenario_colors,
            )

            # Set labels and title
            plt.xlabel("Year")
            plt.ylabel(data["unit"].iloc[0])  # Set the y-axis label to the unit
            # Set y-axis lower limit to 0
            ax.set_ylim(min(0,ax.get_ylim()[0]), ax.get_ylim()[1])
            # Mid point of left and right x-positions
            mid = (fig.subplotpars.right + fig.subplotpars.left)/2
            if title_dict:
                plt.suptitle(title_dict[variable], x=0.4)  # Set the title to the variable
                plt.title(region)
            else:
                fig.suptitle(variable, fontsize=16)
            # Show legend and grid
            plt.legend(title="Legend", bbox_to_anchor=(1.3, 1))
            plt.grid(True)
            # Adjust layout to ensure legend fits within the saved image
            plt.tight_layout()
            # root_variable = variable.split("|")[0].replace(" ", "_")
            if path_dict:
                variable_name = variable.replace(" > 25 μg/m3", "").replace("|", "_").replace(" ", "_").replace("/", "")
                plt.savefig(
                    f"../../../plots/SOD/chpt_20/Regional/{regions_dict[region]}/{path_dict[variable]}/{variable_name}.png"
                )
                plt.close()

In [12]:
harmonized_df = pyam.concat([harmonized_df, tech_diff, life_diff])
harmonized_df = harmonized_df.rename(scenario={
    'REF-v2':'REF', 
    'TECH-TP-v2':'TECH-TP', 
    'LIFE-TP-v2':'LIFE-TP'})

In [13]:
path_dict = {
    'Population|Calories Per Capita|Coefficient of Variation': 'Socio_Economics',
    'Labor Productivity': 'Socio_Economics',
    'Labor Productivity|Agriculture': 'Socio_Economics',
    'Labor Productivity|Energy': 'Socio_Economics',
    'Labor Productivity|Materials': 'Socio_Economics',
    'Labor Productivity|Manufactures': 'Socio_Economics',
    'Labor Productivity|Services': 'Socio_Economics',
    'Labor Productivity|ICTech': 'Socio_Economics',
    'Population|Malnourished': 'Socio_Economics',
    'Population|Poverty Under $2.15': 'Socio_Economics',
    'Population|Shared Sanitation Access': 'Socio_Economics',
    'Population|Safe Water Access': 'Socio_Economics',
    "Health|Deaths": 'Socio_Economics',
    "Health|Deaths|Injuries": 'Socio_Economics',
    "Health|Deaths|Non-Communicable": 'Socio_Economics',
    "Health|Deaths|Communicable": 'Socio_Economics',
    'Health|Deaths (Difference Relative to REF)':'Socio_Economics',
    'Health|Deaths|Communicable (Difference Relative to REF)':'Socio_Economics',
    'Health|Deaths|Injuries (Difference Relative to REF)':'Socio_Economics',
    'Health|Deaths|Non-Communicable (Difference Relative to REF)':'Socio_Economics',}

title_dict = {
    'Population|Calories Per Capita|Coefficient of Variation': 'Calories Per Capita Coefficient of Variation',
    'Labor Productivity': 'Labor Productivity',
    'Labor Productivity|Agriculture': 'Labor Productivity Agriculture',
    'Labor Productivity|Energy': 'Labor Productivity|Energy',
    'Labor Productivity|Materials': 'Labor Productivity|Materials',
    'Labor Productivity|Manufactures': 'Labor Productivity|Manufactures',
    'Labor Productivity|Services': 'Labor Productivity|Services',
    'Labor Productivity|ICTech': 'Labor Productivity|ICTech',
    'Population|Malnourished': 'Population|Malnourished',
    'Population|Poverty Under $2.15': 'Population|Poverty Under $2.15',
    'Population|Shared Sanitation Access': 'Population|Shared Sanitation Access',
    'Population|Safe Water Access': 'Population|Safe Water Access',
    "Health|Deaths": 'Deaths',
    "Health|Deaths|Injuries": 'Deaths due to Injuries',
    "Health|Deaths|Non-Communicable": 'Deaths due to Non-Communicable Disease',
    "Health|Deaths|Communicable": 'Deaths due to Communicable Disease',
    'Health|Deaths (Difference Relative to REF)':'Deaths (Difference Relative to REF)',
    'Health|Deaths|Communicable (Difference Relative to REF)':'Deaths due to Communicable Disease (Difference Relative to REF)',
    'Health|Deaths|Injuries (Difference Relative to REF)':'Deaths due to Injuries (Difference Relative to REF)',
    'Health|Deaths|Non-Communicable (Difference Relative to REF)':'Deaths due to Non-Communicable Disease (Difference Relative to REF)',}

# plot_data_individual_figures(harmonized_df, title_dict=title_dict, path_dict=path_dict)

In [14]:
harmonized_df.to_excel('../../../data/SOD/model_results/to_share/IFs_to_share.xlsx')